# Music Recomendations with XGBoost

Using Gradient Boosted Trees to Provide Music Recommendations


This notebook will NOT be part of the workshop. This is used to pretrain the xgboost music recommendation model. Trained model should be uploaded to the correct S3 bucket to be used for deployment


This notebook was created and tested on an ml.m4.xlarge TODO : Check notebook instance.



# Data

Explain Million Song Data

In [70]:
%matplotlib inline

import pandas as pd
import numpy as np
import time

## Data Preparation : user-song-interaction data.
(Skip this section if you are already using this)

In [193]:
##TODO : Make sure
data = pd.read_csv("data_75000.csv", sep=",", header = 0)

In [ ]:
#triplets_file = 'https://static.turi.com/datasets/millionsong/10000.txt'
songs_metadata_file = 'https://static.turi.com/datasets/millionsong/song_data.csv'

In [190]:
#data = pd.read_csv(triplets_file, sep="\t", header = None)
#data.columns = ['user_id', 'song_id', 'listen_count']

#Read song  metadata
song_meta_data_df =  pd.read_csv(songs_metadata_file)

In [191]:
len(song_meta_data_df)

1000000

In [192]:
song_df_2.head()

,song_id,short_song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,1228.0,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,7776.0,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,4460.0,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,9137.0,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,5903.0,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [ ]:
data

In [ ]:
##No need to use it if already using data_75000.csv

#Create dictionary of unique users 
users = data['user_id']
unique_users = set(users)
user_dict = {}

i=1
for user in unique_users:
    user_dict[i] = user
    i = i + 1

key=18
print(user_dict[key])    

In [ ]:
##No need to use it if already using data_75000.csv
#Create dictionary of unique songs 
songs = data['song_id']
unique_songs = set(songs)
song_dict = {}

i=1
for song in unique_songs:
    song_dict[i] = song
    i = i + 1

key=20
print(song_dict[key])   

In [ ]:
##No need to use it if already using data_75000.csv
print("Number of unique users in the dictionary : " , len(user_dict))
all_users_df = pd.DataFrame.from_dict(user_dict, orient='index', columns=['long_user_id'])
print("Number of unique users in the data frame : " , len(all_users_df))

print("Number of unique songs in the dictionary : " , len(song_dict))
all_songs_df = pd.DataFrame.from_dict(song_dict, orient='index', columns=['long_song_id'])
print("Number of unique songs in the data frame : " , len(all_songs_df))

print(all_users_df)
all_users_df = pd.DataFrame.from_dict(user_dict, orient='index',
                       #columns=['short_user_id', 'long_user_id'])

def get_short_user_id(user_id):
    user_match = all_users_df.loc[all_users_df['long_user_id'] == user_id]
    short_user_id = user_match.index.values[0]
    return short_user_id
    
def get_short_song_id(song_id):
    #print("getting short song id for ", song_id)
    #for short_song_id, long_song_id in song_dict.items(): 
     #   if long_song_id == song_id:
      #      return short_song_id  
    song_match = all_songs_df.loc[all_songs_df['long_song_id'] == song_id]
    short_song_id = song_match.index.values[0]
    return short_song_id


print("Short User Id : " , get_short_user_id('b80344d063b5ccb3212f76538f3d9e43d87dca9e'))
print("Short User Id : " , get_short_user_id('d8bfd4ec88f0f3773a9e022e3c1a0f1d3b7b6a92'))

print("Short Song Id : " , get_short_song_id('SOAKIMP12A8C130995'))
print("Short Song Id : " , get_short_song_id('SOBXHDL12A81C204C0'))
      

In [ ]:
##No need to use it if already using data_75000.csv
#for i in range(len(data)) : 
for i in range(0,1999999) : 
    long_user_id=data.loc[i, "user_id"]
    short_user_id=get_short_user_id(long_user_id)
    data.loc[i, "short_user_id"] = int(short_user_id)
    
    long_song_id=data.loc[i, "song_id"]
    short_song_id=get_short_song_id(long_song_id)
    data.loc[i, "short_song_id"] = int(short_song_id)

data

In [181]:
len(data)

2000000

In [182]:
data.columns

Index(['listen_count', 'short_user_id', 'short_song_id', 'year'], dtype='object')

In [185]:
##Get the short_song_id column from data dataframe
short_song_id_col = data['short_song_id']
short_song_id_col

##Add to song dataframe
song_df_2.insert(1,'short_song_id', short_song_id_col)

In [186]:
song_df_2.head()

,song_id,short_song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,1228.0,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,7776.0,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,4460.0,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,9137.0,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,5903.0,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [187]:
song_df_2.to_csv("song_metadata.csv")

## Lets use the prepared music data here.

In [88]:
data = pd.read_csv("data_75000.csv", sep=",", header = 0)

In [89]:
data.drop('Unnamed: 0', axis=1, inplace=True)

data.columns = ['user_id', 'song_id', 'listen_count','short_user_id', 'short_song_id']

In [90]:
len(data)

2000000

In [ ]:
data.head(100)

In [91]:
##Combine the user and song dataframes to get the complete data set.
#Iterate through the dataframe
##Not the most efficient code.  Can use better utilities
##Only going to use 10K rows to speed up.
for i, row in data.iterrows():
    if(i < 1000) :
        #print("row is ", type(row)#, " : ", row)
        user_id = row['user_id']
        song_id = row['song_id']
        
        song_match = song_df_2.loc[song_df_2['song_id'] == song_id]
        matched_song_year = song_match['year']

        if len(song_match) != 0 : 
            print("i is : " , i , "Matching song year is : " , song_match['year'].values[0])
            data.at[i,"year"] = song_match['year'].values[0]
            #data.insert(i, "title", song_match['title'].values[0])
    else :
        break

i is :  0 Matching song year is :  0
i is :  1 Matching song year is :  1976
i is :  2 Matching song year is :  2007
i is :  3 Matching song year is :  2005
i is :  4 Matching song year is :  1999
i is :  5 Matching song year is :  2007
i is :  6 Matching song year is :  2008
i is :  7 Matching song year is :  1999
i is :  8 Matching song year is :  0
i is :  9 Matching song year is :  2002
i is :  10 Matching song year is :  2007
i is :  11 Matching song year is :  0
i is :  12 Matching song year is :  1989
i is :  13 Matching song year is :  0
i is :  14 Matching song year is :  0
i is :  15 Matching song year is :  2009
i is :  16 Matching song year is :  0
i is :  17 Matching song year is :  2007
i is :  18 Matching song year is :  1999
i is :  19 Matching song year is :  2008
i is :  20 Matching song year is :  2008
i is :  21 Matching song year is :  2009
i is :  22 Matching song year is :  2009
i is :  23 Matching song year is :  0
i is :  24 Matching song year is :  2007
i is :

i is :  201 Matching song year is :  0
i is :  202 Matching song year is :  2004
i is :  203 Matching song year is :  2007
i is :  204 Matching song year is :  2007
i is :  205 Matching song year is :  2007
i is :  206 Matching song year is :  0
i is :  207 Matching song year is :  0
i is :  208 Matching song year is :  1993
i is :  209 Matching song year is :  2003
i is :  210 Matching song year is :  2007
i is :  211 Matching song year is :  0
i is :  212 Matching song year is :  2000
i is :  213 Matching song year is :  2000
i is :  214 Matching song year is :  1978
i is :  215 Matching song year is :  2003
i is :  216 Matching song year is :  2009
i is :  217 Matching song year is :  2000
i is :  218 Matching song year is :  2009
i is :  219 Matching song year is :  0
i is :  220 Matching song year is :  2001
i is :  221 Matching song year is :  2009
i is :  222 Matching song year is :  2007
i is :  223 Matching song year is :  1996
i is :  224 Matching song year is :  2007
i is : 

i is :  401 Matching song year is :  1995
i is :  402 Matching song year is :  2008
i is :  403 Matching song year is :  2006
i is :  404 Matching song year is :  2006
i is :  405 Matching song year is :  2006
i is :  406 Matching song year is :  2007
i is :  407 Matching song year is :  2004
i is :  408 Matching song year is :  2007
i is :  409 Matching song year is :  2006
i is :  410 Matching song year is :  2007
i is :  411 Matching song year is :  2003
i is :  412 Matching song year is :  2005
i is :  413 Matching song year is :  2009
i is :  414 Matching song year is :  0
i is :  415 Matching song year is :  2008
i is :  416 Matching song year is :  2005
i is :  417 Matching song year is :  0
i is :  418 Matching song year is :  2005
i is :  419 Matching song year is :  2005
i is :  420 Matching song year is :  2010
i is :  421 Matching song year is :  2009
i is :  422 Matching song year is :  0
i is :  423 Matching song year is :  2005
i is :  424 Matching song year is :  2006
i

i is :  598 Matching song year is :  2005
i is :  599 Matching song year is :  2010
i is :  600 Matching song year is :  2007
i is :  601 Matching song year is :  2008
i is :  602 Matching song year is :  2008
i is :  603 Matching song year is :  2007
i is :  604 Matching song year is :  2002
i is :  605 Matching song year is :  2005
i is :  606 Matching song year is :  2000
i is :  607 Matching song year is :  2010
i is :  608 Matching song year is :  2005
i is :  609 Matching song year is :  0
i is :  610 Matching song year is :  2008
i is :  611 Matching song year is :  2001
i is :  612 Matching song year is :  2004
i is :  613 Matching song year is :  2009
i is :  614 Matching song year is :  0
i is :  615 Matching song year is :  2008
i is :  616 Matching song year is :  1997
i is :  617 Matching song year is :  0
i is :  618 Matching song year is :  2005
i is :  619 Matching song year is :  2008
i is :  620 Matching song year is :  2008
i is :  621 Matching song year is :  1990
i

i is :  799 Matching song year is :  1972
i is :  800 Matching song year is :  2008
i is :  801 Matching song year is :  2004
i is :  802 Matching song year is :  1979
i is :  803 Matching song year is :  2010
i is :  804 Matching song year is :  1990
i is :  805 Matching song year is :  2004
i is :  806 Matching song year is :  1989
i is :  807 Matching song year is :  2002
i is :  808 Matching song year is :  2001
i is :  809 Matching song year is :  2009
i is :  810 Matching song year is :  2004
i is :  811 Matching song year is :  0
i is :  812 Matching song year is :  2008
i is :  813 Matching song year is :  2007
i is :  814 Matching song year is :  1988
i is :  815 Matching song year is :  2007
i is :  816 Matching song year is :  2004
i is :  817 Matching song year is :  2003
i is :  818 Matching song year is :  2009
i is :  819 Matching song year is :  0
i is :  820 Matching song year is :  1986
i is :  821 Matching song year is :  0
i is :  822 Matching song year is :  1997
i

i is :  999 Matching song year is :  1995


In [92]:
print("After update")
data.head(10)

After update


,user_id,song_id,listen_count,short_user_id,short_song_id,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,30544.0,1228.0,0.0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,30544.0,7776.0,1976.0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,30544.0,4460.0,2007.0
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,30544.0,9137.0,2005.0
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,30544.0,5903.0,1999.0
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5,30544.0,8287.0,2007.0
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODXRTY12AB0180F3B,1,30544.0,7607.0,2008.0
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFGUAY12AB017B0A8,1,30544.0,8774.0,1999.0
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFRQTD12A81C233C0,1,30544.0,1754.0,0.0
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOHQWYZ12A6D4FA701,1,30544.0,1796.0,2002.0


In [93]:
data.to_csv("music_data_song_year_with_long_ids.csv")

In [94]:
#Drop the long ids, not needed anymore.
data.drop('user_id', axis=1, inplace=True)
data.drop('song_id', axis=1, inplace=True)

In [95]:
data.head(10)

,listen_count,short_user_id,short_song_id,year
0,1,30544.0,1228.0,0.0
1,2,30544.0,7776.0,1976.0
2,1,30544.0,4460.0,2007.0
3,1,30544.0,9137.0,2005.0
4,1,30544.0,5903.0,1999.0
5,5,30544.0,8287.0,2007.0
6,1,30544.0,7607.0,2008.0
7,1,30544.0,8774.0,1999.0
8,1,30544.0,1754.0,0.0
9,1,30544.0,1796.0,2002.0


In [96]:
data.to_csv("music_data_song_year.csv")

In [78]:
##NOT USING THE REST OF THE CELLs.  GO TO SETUP DIRECTLY

In [ ]:
#song_df =pd.merge(data, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")
#song_df = pd.merge(data, song_df_2)
#len(song_df)

In [ ]:
#song_df = song_df.head(10000)

#Merge song title and artist_name columns to make a merged column
#song_df['song'] = song_df['title'].map(str) + " - " + song_df['artist_name']
#len(song_df)

In [ ]:
#song_df.head(100000)

In [ ]:
#song_df['user_id'] = np.arange(song_df.shape[0])
#len(song_df)

In [ ]:
#song_df.head(1000)


In [ ]:
#song_df['song_id'] = np.arange(song_df.shape[0])

In [ ]:
#song_df.head(5000)

#len(song_df)


In [ ]:
#song_df.loc[300]

In [ ]:
##For SageMaker XGBoost, the predictor variable should be the first column and there should be no headers in the file.
##So move the 'listen_count' column to the begining of the dataframe.
#rating = song_df['listen_count']
#song_df.drop(labels=['listen_count'], axis=1,inplace = True)
#song_df.insert(0, 'listen_count', rating)
#song_df

In [ ]:
# One hot encode the 'song' attribute 
##MAY BE NOT NEEDED
#song_df = pd.concat([song_df,pd.get_dummies(song_df['song'], prefix='song')],axis=1)
#song_df

In [ ]:
##Drop the original feature, since it is not needed anymore.
#song_df.drop(['song'],axis=1, inplace=True)

In [ ]:
#song_df

In [ ]:
##Drop the original features, since it is not needed anymore.
#song_df.drop(['title'],axis=1, inplace=True)
#song_df.drop(['release'],axis=1, inplace=True)
#song_df.drop(['artist_name'],axis=1, inplace=True)

In [ ]:
#len(song_df.columns)

In [ ]:
#song_df.columns

# Training Setup

This notebook was created and tested on an ml.p2.xlarge TODO : Check notebook instance.


Specify the below-

    The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
    The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).


In [97]:
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()


In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

In [99]:
#bucket = 'reinvent2019-sagemaker'  ##TODO : Change this to session bucket
bucket = 'reinvent-sagemaker-deployment-2019'
prefix = 'sagemaker/recommendations-xgboost-songsnew'



And now let's split the data into training, validation, and test sets. This will help prevent us from overfitting the model, and allow us to test the models accuracy on data it hasn't already seen.

In [102]:
data_subset = data[0:999]

In [105]:
train_data, validation_data, test_data = np.split(data_subset.sample(frac=1, random_state=150), [int(0.7 * len(data_subset)), int(0.9 * len(data_subset))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)
test_data.to_csv('test.csv', header=False, index=False)

data_subset.to_csv('music_users_items_for_predictions.csv', header=True, index=False)


print("Number of training samples : " , len(train_data))
print("Number of validation samples : " , len(validation_data))
print("Number of test samples : " , len(test_data))



Number of training samples :  699
Number of validation samples :  200
Number of test samples :  100


In [106]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'predictions/music_users_items_for_predictions.csv')).upload_file('music_users_items_for_predictions.csv')

# Train

Moving onto training, first we'll need to specify the locations of the XGBoost algorithm containers.

In [107]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

	get_image_uri(region, 'xgboost', '0.90-1').


In [108]:

import sagemaker
from sagemaker.predictor import csv_serializer


s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')


In [109]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=6,
                        eta=0.2,
                        gamma=5,
                        min_child_weight=6,
                        subsample=0.9,
                        silent=0,
                        objective='reg:linear',
                        num_round=60)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2019-12-02 20:20:08 Starting - Starting the training job...
2019-12-02 20:20:09 Starting - Launching requested ML instances......
2019-12-02 20:21:12 Starting - Preparing the instances for training...
2019-12-02 20:22:00 Downloading - Downloading input data...
2019-12-02 20:22:33 Training - Downloading the training image...
2019-12-02 20:23:04 Uploading - Uploading generated training model
2019-12-02 20:23:04 Completed - Training job completed
Arguments: train
[2019-12-02:20:22:53:INFO] Running standalone xgboost training.
[2019-12-02:20:22:53:INFO] File size need to be processed in the node: 0.02mb. Available memory size in the node: 8528.95mb
[2019-12-02:20:22:53:INFO] Determined delimiter of CSV input is ','
[20:22:53] S3DistributionType set as FullyReplicated
[20:22:53] 699x3 matrix with 2097 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2019-12-02:20:22:53:INFO] Determined delimiter of CSV input is ','
[20:22:53] S3DistributionType set as Full

Training seconds: 64
Billable seconds: 64


# Host

Now that we've trained the algorithm, let's create a model and deploy it to a hosted endpoint.

In [110]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')


---------------------------------------------------------------------------------------------------!

# Evaluate

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making an http POST request. But first, we'll need to setup serializers and deserializers for passing our test_data NumPy arrays to the model behind the endpoint.


In [111]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

In [112]:
print("test_data type is ", type(test_data))

ratings = test_data['listen_count']

print("ratings type ", type(ratings))

test_data.drop('listen_count', axis=1, inplace=True)

test_data type is  <class 'pandas.core.frame.DataFrame'>
ratings type  <class 'pandas.core.series.Series'>


In [114]:
##Indice causing errors : 48,75,95.134
test_data_matrix = test_data.as_matrix()

#Try removing these indices : TODO

print("test_data_matrix type is ", type(test_data_matrix), " shape ", test_data_matrix.shape)

test_data_matrix_subset = test_data_matrix[:10]

predictions=[]

for i in range(0, 20):
    print('Inference traffic ', test_data_matrix[i])
    predicted_value = xgb_predictor.predict(test_data_matrix[i])
    predictions.append(predicted_value)
    print("predicted value ", predicted_value)
    
print("Number of predictions ", len(predictions))
print("Number of original ratings ", len(ratings))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


test_data_matrix type is  <class 'numpy.ndarray'>  shape  (100, 3)
Inference traffic  [54250.  2564.     0.]
predicted value  b'1.44649767876'
Inference traffic  [54250.  2434.  2006.]
predicted value  b'2.18136382103'
Inference traffic  [64566.  4372.  1996.]
predicted value  b'2.14015817642'
Inference traffic  [54250.  8563.  2008.]
predicted value  b'3.86827635765'
Inference traffic  [30544.  3476.  2006.]
predicted value  b'1.69786965847'
Inference traffic  [20457.  8310.  1999.]
predicted value  b'7.45636463165'
Inference traffic  [64566.  3470.  2005.]
predicted value  b'2.19113588333'
Inference traffic  [54250.  7436.  2008.]
predicted value  b'2.63249325752'
Inference traffic  [17970.  7006.  2009.]
predicted value  b'2.22602987289'
Inference traffic  [64566.  7627.     0.]
predicted value  b'2.41111588478'
Inference traffic  [54250.  3214.  2001.]
predicted value  b'1.38932180405'
Inference traffic  [41226.   990.  2005.]
predicted value  b'1.33304142952'
Inference traffic  [3

In [115]:
##Compare with the original values 
for i in range(0, 20):
    #Prediction returned is a byte array.  Convert this to float to compare with the original
    prediction = float(predictions[i].decode())  
    print("predicted value ", prediction, " original value ", ratings.values[i])

predicted value  1.44649767876  original value  4
predicted value  2.18136382103  original value  11
predicted value  2.14015817642  original value  1
predicted value  3.86827635765  original value  2
predicted value  1.69786965847  original value  2
predicted value  7.45636463165  original value  1
predicted value  2.19113588333  original value  1
predicted value  2.63249325752  original value  1
predicted value  2.22602987289  original value  1
predicted value  2.41111588478  original value  1
predicted value  1.38932180405  original value  1
predicted value  1.33304142952  original value  2
predicted value  1.40245342255  original value  2
predicted value  7.2886967659  original value  1
predicted value  2.28048467636  original value  1
predicted value  2.6195294857  original value  1
predicted value  1.3242123127  original value  1
predicted value  2.51563429832  original value  2
predicted value  3.23561954498  original value  9
predicted value  3.02306675911  original value  6


In [149]:
song_df = pd.read_csv(LOCAL_DATA_DIR+"/"+'music_data_song_year_with_long_ids.csv', delimiter =',', encoding='latin-1')

#movie_df.columns = ["movie id", "movie title", "release date", "video release date",
 #             "IMDb URL", "unknown", "Action", "Adventure", "Animation",
  #            "Children's","Comedy","Crime","Documentary","Drama","Fantasy",
   #           "Film-Noir","Horror","Musical","Mystery", "Romance","Sci-Fi",
    #          "Thriller","War","Western"]

#data.drop('Unnamed: 0', axis=1, inplace=True)

#data.columns = ['user_id', 'song_id', 'listen_count','short_user_id', 'short_song_id']

#data = pd.read_csv("data_75000.csv", sep=",", header = 0)


In [164]:
song_df.head(100)

,Unnamed: 0,user_id,song_id,listen_count,short_user_id,short_song_id,year
0,0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,30544.0,1228.0,0.0
1,1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,30544.0,7776.0,1976.0
2,2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,30544.0,4460.0,2007.0
3,3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,30544.0,9137.0,2005.0
4,4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,30544.0,5903.0,1999.0
5,5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5,30544.0,8287.0,2007.0
6,6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODXRTY12AB0180F3B,1,30544.0,7607.0,2008.0
7,7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFGUAY12AB017B0A8,1,30544.0,8774.0,1999.0
8,8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFRQTD12A81C233C0,1,30544.0,1754.0,0.0
9,9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOHQWYZ12A6D4FA701,1,30544.0,1796.0,2002.0


In [167]:
song_match = song_df.loc[song_df['short_song_id'] == 5128.0]

song_match

,Unnamed: 0,user_id,song_id,listen_count,short_user_id,short_song_id,year
11,11,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOIZAZL12A6701C53B,5,30544.0,5128.0,0.0
7730,7730,390c2e81bc9cf885608a0891c0a7eb13f1fd3336,SOIZAZL12A6701C53B,1,75049.0,5128.0,NaN
11210,11210,2a0bfd858b2ed706d390d5b6562b8fafe26092ba,SOIZAZL12A6701C53B,1,27585.0,5128.0,NaN
11352,11352,77448d704c1b9acc37037d400302ce7f8a9496c7,SOIZAZL12A6701C53B,1,64279.0,5128.0,NaN
11830,11830,6b015fe063d77e86c3b995fe1c18a2fdce9b8946,SOIZAZL12A6701C53B,20,35202.0,5128.0,NaN
13169,13169,e697f7b095a4f7cd83f00575741cf5d1a1850b37,SOIZAZL12A6701C53B,4,45207.0,5128.0,NaN
16784,16784,9c8bdd66608468fe0779a978765aa8760572c99f,SOIZAZL12A6701C53B,11,33191.0,5128.0,NaN
30149,30149,c06d85f48d4c2c9b0d4a4d0adc13a10a1f258216,SOIZAZL12A6701C53B,8,72333.0,5128.0,NaN
31634,31634,65014d7a44af245dc85801c64bbeecbbcdefa701,SOIZAZL12A6701C53B,4,20220.0,5128.0,NaN
32263,32263,25aa0ec046aa788bf5366d7b7bf81c17fa022e46,SOIZAZL12A6701C53B,30,13588.0,5128.0,NaN


In [161]:
song_df_2.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [180]:
##Music recommendations are working
## But we need to provide song_df and song_df_2
## What are the files that are necessary to upload to music_recommendations_data?
##     1. Need song meta data (Can we use both short and long ids here?)
##     2. Inference traffic 'music_users_items_for_predictions.csv'


In [202]:
runtime_client = boto3.client('runtime.sagemaker')

LOCAL_DATA_DIR = "."

def get_recommendations_for_user(user_id, show_predictions):
    predictions_for_user = str(user_id)
    predictions = []
    
    with open(LOCAL_DATA_DIR+"/"+'music_users_items_for_predictions.csv', 'r') as f:
        contents = f.readlines() 
    
    for i in range(0, len(contents) - 1):
        line = contents[i]
        
        #print("line ", line)
        split_data = line.split(',')
        
        #print("split_data ", split_data)
        #Remove the original rating value from data used for prediction
        original_value = split_data.pop(0)
        
        user = split_data[0]
        item = split_data[1]
        

        if (user == predictions_for_user) : 
            
            print('Predicting rating for User ', user, 'for item ', item)

            payload = ','.join(split_data)

            #response = runtime_client.invoke_endpoint(EndpointName=endpoint_name,
             #                                     ContentType='text/csv', 
              #                                    Body=payload)
            
            prediction = xgb_predictor.predict(payload)
            #prediction = response['Body'].read().decode('utf-8')
            predictions.append([item, prediction])

            #print("Original Value ", original_value , "Prediction : ", float(prediction))

    if show_predictions:        
        sorted_predcitions =    sorted(predictions, key = lambda x: x[1], reverse=True)     

        ## Let's show only the top 10 recommendations
        recommendations = sorted_predcitions[0:9]

        print("Recommended songs for user with id : ", predictions_for_user)
        for rec in recommendations: 
            #print("rec is ", type(rec))
            song_id = float(rec[0])
            #print("recommended_song_item ", song_id)
            song_match = song_df_2.loc[song_df['short_song_id'] == song_id]
            song_title = song_match['title'].values[0]
            print("\t", song_title )
            
            

In [203]:
user_id = '30544.0'
get_recommendations_for_user(user_id, True)

Predicting rating for User  30544.0 for item  1228.0
Predicting rating for User  30544.0 for item  7776.0
Predicting rating for User  30544.0 for item  4460.0
Predicting rating for User  30544.0 for item  9137.0
Predicting rating for User  30544.0 for item  5903.0
Predicting rating for User  30544.0 for item  8287.0
Predicting rating for User  30544.0 for item  7607.0
Predicting rating for User  30544.0 for item  8774.0
Predicting rating for User  30544.0 for item  1754.0
Predicting rating for User  30544.0 for item  1796.0
Predicting rating for User  30544.0 for item  6154.0
Predicting rating for User  30544.0 for item  5128.0
Predicting rating for User  30544.0 for item  8.0
Predicting rating for User  30544.0 for item  9621.0
Predicting rating for User  30544.0 for item  3687.0
Predicting rating for User  30544.0 for item  7239.0
Predicting rating for User  30544.0 for item  3355.0
Predicting rating for User  30544.0 for item  208.0
Predicting rating for User  30544.0 for item  8900

In [206]:
##Seems to be working only for one user so far.
user_id = '1334.0'
get_recommendations_for_user(user_id, True)

Recommended songs for user with id :  1334.0
